# 앙상블 검색기(Ensemble Retriever) : 다양한 연습

`EnsembleRetriever`는 여러 검색기를 결합하여 더 강력한 검색 결과를 제공하는 LangChain의 기능입니다. 이 검색기는 다양한 검색 알고리즘의 장점을 활용하여 단일 알고리즘보다 더 나은 성능을 달성할 수 있습니다.

**주요 특징**
1. 여러 검색기 통합: 다양한 유형의 검색기를 입력으로 받아 결과를 결합합니다.
2. 결과 재순위화: [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) 알고리즘을 사용하여 결과의 순위를 조정합니다.
3. 하이브리드 검색: 주로 `sparse retriever`(예: BM25)와 `dense retriever`(예: 임베딩 유사도)를 결합하여 사용합니다.

**장점**
- Sparse retriever: 키워드 기반 검색에 효과적
- Dense retriever: 의미적 유사성 기반 검색에 효과적

이러한 상호 보완적인 특성으로 인해 `EnsembleRetriever`는 다양한 검색 시나리오에서 향상된 성능을 제공할 수 있습니다.

자세한 내용은 [LangChain 공식 문서](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)를 참조하세요.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH10-Retriever")

- `EnsembleRetriever`를 초기화하여 `BM25Retriever`와 `FAISS` 검색기를 결합합니다. 각 검색기의 가중치를 설정됩니다.


In [8]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


# 샘플 문서 리스트
doc_list = [
    "I like apples",
    "I like apple company",
    "I like apple's iphone",
    "Apple is my favorite company",
    "I like apple's ipad",
    "I like apple's macbook",
]


# bm25 retriever와 faiss retriever를 초기화합니다.
bm25_retriever = BM25Retriever.from_texts(
    doc_list,
)
bm25_retriever.k = 1  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

# 임베딩(Embedding) 생성
model_name = "BAAI/bge-m3"
model_kwargs = {"device":"cpu"}
encode_kwargs = {"normalize_embeddings":True}
embeddings = HuggingFaceBgeEmbeddings(
model_name=model_name, model_kwargs=model_kwargs,encode_kwargs=encode_kwargs

)
faiss_vectorstore = FAISS.from_texts(
    doc_list,
    embeddings,
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 1})

# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
)

/home/jun/.conda/envs/chromagroq/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


`ensemble_retriever` 객체의 `get_relevant_documents()` 메서드를 호출하여 관련성 높은 문서를 검색합니다.


In [3]:
# 검색 결과 문서를 가져옵니다.
query = "my favorite fruit is apple"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Apple is my favorite company

Content: I like apples

[BM25 Retriever]
Content: Apple is my favorite company

[FAISS Retriever]
Content: I like apples



In [5]:
# 검색 결과 문서를 가져옵니다.
query = "Apple company makes my favorite iphone"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Apple is my favorite company

[BM25 Retriever]
Content: Apple is my favorite company

[FAISS Retriever]
Content: Apple is my favorite company



## 런타임 Config 변경

런타임에서도 retriever 의 속성을 변경할 수 있습니다. 이는 `ConfigurableField` 클래스를 사용하여 가능합니다.

- `weights` 매개변수를 `ConfigurableField` 객체로 정의합니다.
  - 필드의 ID는 "ensemble_weights"로 설정합니다.


In [6]:
from langchain_core.runnables import ConfigurableField


ensemble_retriever = EnsembleRetriever(
    # 리트리버 목록을 설정합니다. 여기서는 bm25_retriever와 faiss_retriever를 사용합니다.
    retrievers=[bm25_retriever, faiss_retriever],
).configurable_fields(
    weights=ConfigurableField(
        # 검색 매개변수의 고유 식별자를 설정합니다.
        id="ensemble_weights",
        # 검색 매개변수의 이름을 설정합니다.
        name="Ensemble Weights",
        # 검색 매개변수에 대한 설명을 작성합니다.
        description="Ensemble Weights",
    )
)

- 검색 시 `config` 매개변수를 통해 검색 설정을 지정합니다.
  - `ensemble_weights` 옵션의 가중치를 [1, 0]으로 설정하여 **모든 검색 결과의 가중치가 BM25 retriever 에 더 많이 부여** 되도록 합니다.

In [7]:
config = {"configurable": {"ensemble_weights": [1, 0]}}

# config 매개변수를 사용하여 검색 설정을 지정합니다.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # 검색 결과인 docs를 출력합니다.

[Document(page_content='Apple is my favorite company'),
 Document(page_content='I like apples')]

이번에는 검색시 모든 검색 결과의 가중치가 **FAISS retriever 에 더 많이 부여** 되도록 합니다.

In [8]:
config = {"configurable": {"ensemble_weights": [0, 1]}}

# config 매개변수를 사용하여 검색 설정을 지정합니다.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # 검색 결과인 docs를 출력합니다.

[Document(page_content='I like apples'),
 Document(page_content='Apple is my favorite company')]

# 한글 형태소 + BM25

In [9]:
def pretty_print(docs):
    for i, doc in enumerate(docs):
        if "score" in doc.metadata:
            print(f"[{i+1}] {doc.page_content} ({doc.metadata['score']:.4f})")
        else:
            print(f"[{i+1}] {doc.page_content}")


In [11]:
# 필요한 라이브러리 설치
# !pip install -qU kiwipiepy konlpy langchain-teddynote

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

In [2]:
kiwi.tokenize("안녕하세요? 형태소 분석기 키위입니다")


[Token(form='안녕', tag='NNG', start=0, len=2),
 Token(form='하', tag='XSA', start=2, len=1),
 Token(form='세요', tag='EF', start=3, len=2),
 Token(form='?', tag='SF', start=5, len=1),
 Token(form='형태소', tag='NNG', start=7, len=3),
 Token(form='분석기', tag='NNG', start=11, len=3),
 Token(form='키위', tag='NNG', start=15, len=2),
 Token(form='이', tag='VCP', start=17, len=1),
 Token(form='ᆸ니다', tag='EF', start=17, len=3)]

In [13]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings

# docs = [
#     Document(
#         page_content="금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다."
#     ),
#     Document(
#         page_content="금융저축보험은 규칙적인 저축을 통해 목돈을 마련할 수 있으며, 생명보험 기능도 겸비하고 있습니다."
#     ),
#     Document(
#         page_content="저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다."
#     ),
#     Document(
#         page_content="금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다."
#     ),
#     Document(
#         page_content="금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다."
#     ),
#     Document(
#         page_content="금보험은 저축성과를 극대화합니다. 특히 노후 대비 저축에 유리하게 구성되어 있습니다."
#     ),
#     Document(
#         page_content="금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요."
#     ),
# ]

In [14]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# 디렉토리로더 초기화
loader = DirectoryLoader("../data", glob="**/*.pdf",loader_cls=PDFPlumberLoader, show_progress=True)
    # ./는 현재 디렉토리를 의미하고, 그 하위에 있는 'data' 폴더
    # glob는 파일을 검색할 때 사용할 패턴   
    # **/는 모든 하위 디렉토리
    # *.pdf는 확장자가 .pdf인 모든 파일

# 문서 로드
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

split_docs = text_splitter.split_documents(docs)

 80%|████████  | 8/10 [00:14<00:03,  1.94s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f9b9ef08e50>>
Traceback (most recent call last):
  File "/home/jun/.conda/envs/chromagroq/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
100%|██████████| 10/10 [00:19<00:00,  1.96s/it]


In [17]:
for split_doc in split_docs:
    print(" ".join([token.form for token in kiwi.tokenize(split_doc.page_content)]))

In [18]:
# 토큰화 함수를 생성
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [19]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device":"cpu"}
encode_kwargs = {"normalize_embeddings":True}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs,encode_kwargs=encode_kwargs)

In [20]:
bm25 = BM25Retriever.from_documents(split_docs)

kiwi_bm25 = BM25Retriever.from_documents(split_docs, preprocess_func=kiwi_tokenize)

faiss = FAISS.from_documents(split_docs, embeddings).as_retriever()

bm25_faiss_73 = EnsembleRetriever(
    retrievers=[bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
bm25_faiss_37 = EnsembleRetriever(
    retrievers=[bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.3, 0.7],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
kiwibm25_faiss_73 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.3, 0.7],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)

retrievers = {
    # "bm25": bm25,
    # "kiwi_bm25": kiwi_bm25,
    # "faiss": faiss,
    # "bm25_faiss_73": bm25_faiss_73,
    # "bm25_faiss_37": bm25_faiss_37,
    # "kiwi_bm25_faiss_73": kiwibm25_faiss_73,
    "kiwi_bm25_faiss_37": kiwibm25_faiss_37,
}

KeyboardInterrupt: 

In [26]:
%time
kiwi_bm25 = BM25Retriever.from_documents(split_docs, preprocess_func=kiwi_tokenize)

CPU times: user 4 μs, sys: 0 ns, total: 4 μs
Wall time: 7.87 μs


In [24]:
%%time
faiss = FAISS.from_documents(split_docs, embeddings).as_retriever()

CPU times: user 1h 8min 39s, sys: 11min 30s, total: 1h 20min 9s
Wall time: 13min 28s


In [27]:
%%time
kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.3, 0.7],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)

CPU times: user 5.22 ms, sys: 0 ns, total: 5.22 ms
Wall time: 5.13 ms


In [28]:
ans = kiwibm25_faiss_37.invoke("임차한 집에 거주하던 중 제 돈으로 집수리를 했습니다. 나중에 돌려받을 수 있나요?")

In [30]:
for a in ans:
    print(a.page_content)

인이 이로 인하여 임차의 목적을 달성할 수 없는 때에는 계약을 해지할 수 있다.
제626조(임차인의 상환청구권) ①임차인이 임차물의 보존에 관한 필요비를 지출한 때에는 임대인에 대하여 그 상환을
청구할 수 있다.
②임차인이 유익비를 지출한 경우에는 임대인은 임대차종료시에 그 가액의 증가가 현존한 때에 한하여 임차인의
지출한 금액이나 그 증가액을 상환하여야 한다. 이 경우에 법원은 임대인의 청구에 의하여 상당한 상환기간을 허여
할 수 있다.
[전문개정 2008. 3. 21.]
제10조의2(초과 차임 등의 반환청구) 임차인이 제7조에 따른 증액비율을 초과하여 차임 또는 보증금을 지급하거나 제
7조의2에 따른 월차임 산정률을 초과하여 차임을 지급한 경우에는 초과 지급된 차임 또는 보증금 상당금액의 반환
을 청구할 수 있다.
[본조신설 2013. 8. 13.]
제11조(일시사용을 위한 임대차) 이 법은 일시사용하기 위한 임대차임이 명백한 경우에는 적용하지 아니한다.
[전문개정 2008. 3. 21.]
으로 본다.<개정 2013. 8. 13.>
⑤ 이 법에 따라 임대차의 목적이 된 주택이 매매나 경매의 목적물이 된 경우에는 「민법」 제575조제1항ㆍ제3항 및
같은 법 제578조를 준용한다.<개정 2013. 8. 13.>
⑥ 제5항의 경우에는 동시이행의 항변권(抗辯權)에 관한 「민법」 제536조를 준용한다.<개정 2013. 8. 13.>
[전문개정 2008. 3. 21.]
제3조의2(보증금의 회수) ① 임차인(제3조제2항 및 제3항의 법인을 포함한다. 이하 같다)이 임차주택에 대하여 보증금
제617조(손해배상, 비용상환청구의 기간) 계약 또는 목적물의 성질에 위반한 사용, 수익으로 인하여 생긴 손해배상의
청구와 차주가 지출한 비용의 상환청구는 대주가 물건의 반환을 받은 날로부터 6월내에 하여야 한다.
제7절 임대차
제618조(임대차의 의의) 임대차는 당사자 일방이 상대방에게 목적물을 사용, 수익하게 할 것을 약정하고 상대방이 이에
대하여 차임을 지급할 것을 약정함으로써 

In [10]:
def print_search_results(retrievers, query):
    print(f"Query: {query}")
    for name, retriever in retrievers.items():
        print(f"{name}    \t: {retriever.invoke(query)[0].page_content}")
    print("===" * 20)

In [11]:
print_search_results(retrievers, "금융보험")
print_search_results(retrievers, "금융 보험")
print_search_results(retrievers, "금융저축보험")
print_search_results(retrievers, "축산물 보험")
print_search_results(retrievers, "저축금융보험")
print_search_results(retrievers, "금융보씨 개인정보 조회")

Query: 금융보험
bm25    	: 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.
kiwi_bm25    	: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.
faiss    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
bm25_faiss_73    	: 금보험은 저축성과를 극대화합니다. 특히 노후 대비 저축에 유리하게 구성되어 있습니다.
bm25_faiss_37    	: 금보험은 저축성과를 극대화합니다. 특히 노후 대비 저축에 유리하게 구성되어 있습니다.
kiwi_bm25_faiss_73    	: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.
kiwi_bm25_faiss_37    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
Query: 금융 보험
bm25    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
kiwi_bm25    	: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.
faiss    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
bm25_faiss_73    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
bm25_faiss_37    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
kiwi_bm25_faiss_73    	: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.
kiwi_bm25_faiss_37    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
Query: 금융저축보험
bm25 